![](./resources/MOOC_refdata_RDM_exploration.png)

### Introduction

This notebook demonstrates the different possibilities to explore and download our harmonized reference data, hosted in the [WorldCereal Reference Data Module (RDM)](https://rdm.esa-worldcereal.org/).<br>
Here, we use the dedicated [RDM API](https://ewoc-rdm-api.iiasa.ac.at/swagger/index.html) to interact with the data through Python code.<br>
For quick exploration of the reference data, you can obviously also make use of our [user interface](https://rdm.esa-worldcereal.org/map).

This notebook only covers the processes of inspecting and downloading existing reference data within the WorldCereal RDM.<br>
Data harmonization and upload should be done through the dedicated user interface, which can be accessed by clicking the "Contribute" button, [here](https://rdm.esa-worldcereal.org/).

For more background information on our vision and approach regarding reference data, visit [the reference data section on our project website](https://esa-worldcereal.org/en/reference-data).

For more technical background information, visit [our documentation portal](https://worldcereal.github.io/worldcereal-documentation/rdm/overview.html).

To engage with us on the topic of reference data, reach out on our [user forum](https://forum.esa-worldcereal.org/c/ref-data/6).

### How to run this notebook?

#### Option 1: Run on Terrascope

You can use a preconfigured environment on [**Terrascope**](https://terrascope.be/en) to run the workflows in a Jupyter notebook environment. Just register as a new user on Terrascope or use one of the supported EGI eduGAIN login methods to get started.

Once you have a Terrascope account, you can run this notebook by clicking the button shown below.

<div class="alert alert-block alert-warning">When you click the button, you will be prompted with "Server Options". Make sure to select the "Worldcereal" image here. Did you choose "Terrascope" by accident? Then go to File > Hub Control Panel > Stop my server, and click the link below once again.</div>

<a href="https://notebooks.terrascope.be/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2FWorldCereal%2Fworldcereal-classification&urlpath=lab%2Ftree%2Fworldcereal-classification%2Fnotebooks%2Fworldcereal_RDM_demo.ipynb&branch=main"><img src="https://img.shields.io/badge/run%20RDM%20demo-Terrascope-brightgreen" alt="Run RDM demo" valign="middle"></a>


#### Option 2: Install Locally

If you prefer to install the package locally, you can create the WorldCereal environment using **Conda** or **pip**.

First clone the repository:
```bash
git clone https://github.com/WorldCereal/worldcereal-classification.git
cd worldcereal-classification
```
Next, install the package locally:
- for Conda: `conda env create -f environment.yml`
- for Pip: `pip install .[train,notebooks]`

### Content
  
- [Before you start](###-Before-you-start)
- [1. Browse and explore collections](#1.-Browse-and-explore-collections)
- [2. Download individual collections](#2.-Download-individual-collections)
- [3. Filter collections](#3.-Filter-collections)
- [4. Get crop counts across datasets](#4.-Get-crop-counts-across-datasets)
- [5. Get individual samples across multiple collections](#5.-Get-individual-samples-across-multiple-collections)

### Before you start

Reference data in the WorldCereal RDM are organized in collections.<br>
A collection contains observations derived from a single source and for a single year.<br>

Each collection is characterized by a **data privacy level**, controlling who can access the data:<br>
- *Public datasets* can be accessed by anyone and have been explicitly curated by the WorldCereal consortium;
- *Private datasets* can only be accessed by the user who uploaded the data;
- *Restricted datasets* are private datasets which were approved by the uploader to be used for training the global WorldCereal classification models, but which are not shared publicly.

Anyone is able to access, explore and download public datasets, without the need for any registration or user account.

In order to upload and access private datasets, you need to sign up for a [Copernicus Data Space Ecosystem (CDSE)](https://dataspace.copernicus.eu/) account.<br>
This is completely free of charge!

In [ ]:
# Before we get started, we first initiate an interaction session with the RDM:
from worldcereal.rdm_api import RdmInteraction
rdm = RdmInteraction()

### 1. Browse and explore collections

In this section, we demonstrate how to retrieve a list of available collections and how to get a bit more information about an individual collection.<br>
For now we will focus on public collections only.

In [ ]:
# Get a list of available collections
# (by default, the following method only returns public collections)
collections = rdm.get_collections(include_private=False)

# Extract the collection ID's
ids = [col.id for col in collections]
print(f'Number of collections found: {len(ids)}')
ids

Number of collections found: 96


['2021_ltu_eurocrops_poly_110',
 '2017_mex_cimmyt1_point_111',
 '2020_rw_wapor_1_point_111',
 '2021_eth_adea_poly_110',
 '2017_ssd_esaprojectsen2agri_point_100',
 '2017_ssd_esaprojectsen2agri_poly_100',
 '2017_uga_radiantearth01_poly_110',
 '2018_aut_lpis_poly_110',
 '2017_bel_lpisflanders_poly_110',
 '2018_glo_glance_point_100',
 '2019_glo_glance_point_100',
 '2017_mdg_madrice_point_110',
 '2020_nld_lpis_poly_110',
 '2018_aus_clumcom_poly_110',
 '2020_fra_lpis_poly_110',
 '2020_glo_glance_point_100',
 '2024_nga_iitaoyocrop2024_poly_110',
 '2018_mdg_madrice_point_110',
 '2021_deu_eurocropsnrw_poly_110',
 '2019_aus_clumcom_poly_110',
 '2019_mdg_madrice_point_110',
 '2019_blz_baimssugarcane_poly_110',
 '2019_tha_boku_point_110',
 '2021_deu_eurocropsls_poly_110',
 '2020_mdg_madrice_point_110',
 '2017_fra_lpis_poly_110',
 '2022_aut_lpis_poly_110',
 '2021_mdg_madrice_point_110',
 '2022_mdg_madrice_point_110',
 '2023_mdg_madrice_point_110',
 '2024_mdg_madrice_point_110',
 '2017_aut_lpis_poly

Collection ID's are constructed according to a fixed naming convention:<br>
(year) _ (country/region) _ (identifier) _ (point or poly) _ (information content)<br>
The latter is represented by a numeric code:
- 100: only land cover information
- 110: land cover and crop type information
- 111: land cover, crop type and irrigation information

Find out more on [this page](https://worldcereal.github.io/worldcereal-documentation/rdm/refdata.html#dataset-naming-convention).

Each individual collection is accompanied by a standard set of metadata.
Let's see what basic information is available:

In [ ]:
# We get the first collection
col = collections[0]

# We can now access the collection's metadata
col.print_metadata()


Most of these metadata items speak for themselves. Let's for now have a look at 3 of them:<br>

- The **list of available crop types** mentions which crop types are present within the collection, but does not inform you on the quantity of samples per type. Further down in this notebook you will see how to extract actual sample counts per crop type.<br>

Crop types are indicated using numeric codes, as defined by our [hierarchical land cover/crop type legend](https://artifactory.vgt.vito.be/artifactory/auxdata-public/worldcereal//legend/WorldCereal_LC_CT_legend_latest.pdf).<br>
To ease interpretation of these codes, the following function translates the information into a more human-readable format:

In [ ]:
from worldcereal.utils.legend import translate_ewoc_codes

crop_types = translate_ewoc_codes(col.ewoc_codes)
crop_types

- The **temporal extent** shows the range of observation dates present in the dataset and can give you a rough indication whether this dataset would be useful for you to consider given a specific growing season of interest.<br>

In [ ]:
print(f'Start date: {col.temporal_extent[0]}, End date: {col.temporal_extent[1]}')

- The **spatial extent** provides you with a bounding box in which all observations are contained and gives you an idea about the location of the samples.

To get a better idea on where the dataset is located, we can visualize the spatial extent on a map:

In [ ]:
from worldcereal.rdm_api.rdm_collection import visualize_spatial_extents
visualize_spatial_extents([col])

### 2. Download individual collections

Let's explore which assets are available for each collection.

#### 2.1 The samples (individual observations)

Samples for individual collections can be downloaded as GeoParquet files.
Each sample at minimum holds information on:
- the observed land cover/crop type (attribute: *ewoc_code*)
- the location (point or polygon, captured in the *geometry* attribute)
- the date for which the observation is valid, i.e. the designated crop was present on the field (attribute: *valid_time*)

Large public collections containing many observations have been automatically subsampled into a representative subset, taking into account crop type variability and spatial distribution of the data. As a user, you have the ability to either download this subset, or the full dataset (by specifying the `subset` parameter in the function below).

Let's create a new *download* folder where this notebook is located and download the samples of our example collection:

In [ ]:
dwnld_folder = './download'
# the following function will automatically create the download folder in case it does not exist
parquet_file = rdm.download_collection_geoparquet(col.id, dwnld_folder, subset=True)

The resulting file can be visualized in QGIS or, for a quick view, using the function below:

In [ ]:
from worldcereal.utils.map import visualize_rdm_geoparquet

visualize_rdm_geoparquet(parquet_file)

#### 2.2 Full metadata

We have previously seen a basic set of metadata for our reference data collection.<br>
During the harmonization of public datasets, the WorldCereal moderators are  collecting an extensive set of metadata to fully document both the original dataset and the harmonization steps that have been undertaken.

Most of this metadata is bundled in a metadata Excel file, hosted on the RDM portal. This metadata can either be downloaded as a dictionary, or as an xlsx file:

In [ ]:
metadata_file = rdm.download_collection_metadata(col.id, dwnld_folder)

metadata = rdm.get_collection_metadata(col.id)
metadata

Explore the contents of the metadata.<br>
Note that in addition to a list of available crop types, the extended metadata version also includes crop statistics (i.e. the amount of samples per crop type):

In [ ]:
crop_counts = rdm.get_collection_stats(col.id)
crop_counts

#### 2.3 Harmonization information

Next to the metadata, each PUBLIC dataset is also accompanied by a harmonization document (PDF), including information about all steps that have been undertaken during data curation and harmonization by the WorldCereal moderator, including the translation of original land cover/crop type information to the WorldCereal legend and computation of the dataset confidence score.<br>
Below, we show how to get this document for an individual dataset:

In [ ]:
# Note: this only works for public datasets!
harmonization_file = rdm.download_collection_harmonization_info(col.id, dwnld_folder)

### 3. Filter collections

Now that you have a basic understanding of how reference data is organized in collections and how to access information about and data within individual collections, let's now focus on how to find relevant reference data for your use case.

We have implemented various filtering methods allowing you to identify collections that contain the data you are looking for.

At the moment you can filter based on:
- crop type
- spatial extent
- temporal extent
- access type (public vs private)

In the sections below we demonstrate each of these cases.

#### 3.1 Filter based on crop types

To filter collections on crop type, you need to provide a list of numerical crop type codes.<br>
All collections containing at least one of the mentioned crop types will be returned.<br>

You can either manually enter a list of crop types, or use the application below to select crop types of interest:

In [ ]:
from notebook_utils.croptypepicker import CropTypePicker, DEMO_CROPS

croptypepicker = CropTypePicker(ewoc_codes=DEMO_CROPS)

The croptypepicker application returns a dataframe containing the ewoc_codes you selected. In case you chose to aggregate multiple classes into a single class, this information is shown in the "*new_label*" column. 

In [ ]:
croptypes = croptypepicker.croptypes
croptypes

In [ ]:
# Get a list of public collections containing these crop types
ewoc_codes = list(croptypes.index.values)

collections = rdm.get_collections(ewoc_codes=ewoc_codes)
ids = [col.id for col in collections]
print(f'Number of collections found: {len(ids)}')
ids

#### 3.2 Filter based on location

To retrieve a list of collections intersecting a certain area of interest, you need to provide a spatial bounding box to the `get_collections` function.<br>
Either enter a bounding box manually, or use the below application to draw a bounding box:

In [ ]:
from worldcereal.utils.map import ui_map

map = ui_map()

In [ ]:
bbox = map.get_extent(projection="latlon")

collections = rdm.get_collections(spatial_extent=bbox)

ids = [col.id for col in collections]
print(f'Number of collections found: {len(ids)}')
ids

To manually define a bounding box, enter the desired coordinates below (lat/lon):

In [ ]:
from openeo_gfmap import BoundingBoxExtent

north = 34.79638823
east = -0.34539808
south = 34.45619011
west = -0.91010781

bbox = BoundingBoxExtent(north=north, east=east, south=south, west=west)

#### 3.3 Filter based on temporal extent

Simply define a start and end date for your period of interest:

In [ ]:
from openeo_gfmap import TemporalContext

# We use the year 2020 as an example:
start_date = "2020-01-01"
end_date = "2020-12-31"

temporal_extent = TemporalContext(start_date=start_date, end_date=end_date)

# Access public collections for the specific temporal range
collections = rdm.get_collections(temporal_extent=temporal_extent)
ids = [col.id for col in collections]
print(f'Number of collections found: {len(ids)}')
ids



#### 3.4 Filter based on data privacy

To change the default behaviour of `get_collections` and get (only) private collections, you can set the `include_public` and `include_private` parameters, as demonstrated below.

NOTE that requesting private collections will trigger a login prompt if you are not already logged in.<br>
Simply click the designated link to login to your CDSE account.

Reminder: Uploading your own collections to the RDM can be accomplished by hitting the "Contribute" button on [this page](https://rdm.esa-worldcereal.org/), where you will be guided through the upload procedure.

In [ ]:
private_collections = rdm.get_collections(include_public=False, include_private=True)
private_ids = [col.id for col in private_collections]
print(f'Number of private collections found: {len(private_ids)}')
private_ids

#### 3.5 Your turn!

Use a combination of the filters as presented above to look for collections in Kenya containing samples for maize.
Additionally filter on the year 2021.

How many public maize samples can be found for this country in the RDM?

### 4. Get crop counts across datasets

So far we have gone through the process of first identifying your collections of interest, and then checking the amount of samples for these datasets one by one by downloading each dataset's metadata.<br>

This way of working is quite cumbersome in case you would like to get crop counts over a large area of interest, as potentially many different collections are involved.<br>
Below, we demonstrate how you can request crop statistics for one or multiple crop types over one or multiple collections in one go.

In [ ]:
# Get total count of soybean and spring wheat samples across two collections in Canada:
collection_ids = ['2021_can_aafccropinventory_point_110', '2018_can_aafccropinventory_point_110']
crop_codes = [1106000020, 1101010002] # soybean + spring wheat

counts = rdm.get_crop_counts(ref_ids=collection_ids, ewoc_codes=crop_codes)
counts

Note that in case you do not specify a list of crop types, the statistics for all crop types present in the collections of interest are returned.

In [ ]:
ref_ids = ['2018_sen_jecamcirad_poly_111', '2019_mli_nhicropharvest_poly_110']

counts = rdm.get_crop_counts(ref_ids)
counts

### 5. Get individual samples across multiple collections

Of course we kept the best for last: we implemented a function that allows you to download individual samples across collections, matching your custom search criteria.

There are different ways to filter the samples you wish to download. You can freely combine these different filters in a single request. You will recognize most of these filters from the `get_collections` functionality:

- `ref_ids`: a list of collection id's
- `ewoc_codes`: a list of crop types
- `bbox`: a spatial bounding box
- `temporal_extent`: a temporal range as defined by a start and end date
- `include_public`: whether or not to include public collections
- `include_private`: whether or not to include private collections
- `subset`: if True, the function will only download a subsample of the samples for which the "extract" attribute is 1 or higher. If False (default), all samples matching your search criteria will be downloaded.
- `min_quality_lc`: only download samples for which the land cover quality score is higher than this number
- `min_quality_ct`: only download samples for which the crop type quality score is higher than this number

Below we provide one example of this functionality, but feel free to play around and request samples for your specific area/period/crop types of interest!

For instance, you can make use of the `ui_map` function to draw your own bounding box...

In [ ]:
from openeo_gfmap import BoundingBoxExtent

# supply a bounding box (this one is located on Java, Indonesia)
bbox = BoundingBoxExtent(north=-6.577303, east=111.950684, south=-8.05923, west=107.116699)

# do not explicitly filter on collections
ref_ids = None

# Do not filter on date
temporal_extent = None

# Do not filter on crop type
ewoc_codes = None

# Include public collections only
include_public = True
include_private = False

# Limit sample download to a subset in case many samples available
subset = True

# Minimum quality score for samples
min_quality_ct = 75
min_quality_lc = 0

gdf = rdm.get_samples(
    ref_ids=ref_ids,
    subset=subset,
    spatial_extent=bbox,
    temporal_extent=temporal_extent,
    ewoc_codes=ewoc_codes,
    include_public=include_public,
    include_private=include_private,
    min_quality_ct=min_quality_ct,
    min_quality_lc=min_quality_lc,
)

print(f"Total number of samples downloaded: {len(gdf)}")
gdf.head()

The dataframe you get as a result includes information on the origin of each individual sample in the "*ref_id*" attribute:

In [ ]:
gdf['ref_id'].unique()

You can now save this dataframe as a geoparquet file and visualize it in QGIS or using the same visualization function as before.

Note that in some cases, the resulting dataframe will contain both polygons and points, depending on the collections which were matching your request criteria.

In [ ]:
from pathlib import Path
from worldcereal.utils.map import visualize_rdm_geoparquet

# save as geoparquet file and visualize
dwnld_folder = './download'
parquet_file = str(Path(dwnld_folder) / 'samples.geoparquet')
gdf.to_parquet(parquet_file)

visualize_rdm_geoparquet(parquet_file)


Congratulations, you have reached the end of this exercise!

You have acquired the necessary skills to request the reference data needed to train your own crop type classification algorithms!<br>
In Version 2 of the [WorldCereal processing system](https://github.com/WorldCereal/worldcereal-classification), we will demonstrate how to proceed to actually train your algorithms and produce a crop type map for your region of interest.

In case you did not find the data you are looking for, please consider contributing data to the platform, either as a private, restricted or fully public collection. Read more about our data collection and sharing efforts, [here](https://esa-worldcereal.org/en/reference-data).